# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
project_name = 'fraud-demo-v3'

In [2]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

> 2021-09-14 00:01:26,196 [info] loaded project fraud-demo-v3 from MLRun DB


## Step 1 - Create a Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early_sense sensor data as our spine, so for each early_sense event we will create produce a row in the resulted Feature Vector.

In [3]:
# Define the list of features we will be using
features = ['transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_count_14d', 
            'transactions.es_health_count_14d',
            'transactions.es_otherservices_count_14d', 
            'transactions.es_food_count_14d',
            'transactions.es_hotelservices_count_14d', 
            'transactions.es_barsandrestaurants_count_14d',
            'transactions.es_tech_count_14d', 
            'transactions.es_sportsandtoys_count_14d',
            'transactions.es_wellnessandbeauty_count_14d', 
            'transactions.es_hyper_count_14d',
            'transactions.es_fashion_count_14d', 
            'transactions.es_home_count_14d', 
            'transactions.es_travel_count_14d', 
            'transactions.es_leisure_count_14d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week',
            'events.*']

In [4]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
transactions_fv.save()

## Step 2 - Produce a training dataset <= produce is not the right word

In [5]:
# unclear how this part ties up to the next. the var train_dataset is not used in the following step yet the code is needed, need to explain why

In [6]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector as dataframe and save the dataset to parquet
train_dataset = fstore.get_offline_features(fv_name, target=ParquetTarget())

> 2021-09-14 00:01:27,173 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-v3-alexz/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2021-09-14T00:01:27.173012+00:00', 'size': 2568180}


In [7]:
# Preview our dataset
train_dataset.to_dataframe().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,amount_avg_12h,amount_max_24h,amount_sum_24h,...,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,event_details_change,event_login,event_password_change,label
49995,43.21,73.90,2.0,36.95,43.21,73.90,2.0,36.950000,43.21,-0.21,...,1,0,56,30.69,20,6,1.0,0.0,0.0,0
49996,28.89,28.89,1.0,28.89,28.89,28.89,1.0,28.890000,28.89,28.89,...,0,1,56,28.89,20,6,1.0,0.0,0.0,0
49997,32.67,32.67,1.0,32.67,63.77,208.76,6.0,34.793333,140.28,487.49,...,0,1,127,32.67,20,6,1.0,0.0,0.0,0
49998,16.26,16.26,1.0,16.26,16.26,16.26,1.0,16.260000,59.33,460.61,...,0,1,53,16.26,20,6,0.0,0.0,1.0,0
49999,9.75,9.75,1.0,9.75,17.19,26.94,2.0,13.470000,17.19,39.00,...,1,0,124,9.75,20,6,0.0,0.0,1.0,0


## Step 3 - Train our model with with AutoML on a Cluster

In [ ]:
# explain what is happening, what does automl means, what does it mean 'on a cluster'

In [8]:
from mlrun.platforms import auto_mount

# Import the Sklearn classifier function from the functions hub
classifier_fn = mlrun.import_function('hub://sklearn-classifier')

In [9]:
# Prepare the parameters list for the training function
# We will be using 3 different models
training_params = {"model_name": ['transaction_fraud_rf', 
                             'transaction_fraud_xgboost', 
                             'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
train_task = mlrun.new_task('training', 
                      inputs={'dataset': transactions_fv.uri},
                      params={'label_column': 'label'}
                     )

train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

# Specify our cluster image
classifier_fn.spec.image = 'mlrun/mlrun'

# Run training
classifier_fn.run(train_task, local=False)

> 2021-09-14 00:01:27,523 [info] starting run training uid=c05d2512573e4d1fad969f985ad3e8a2 DB=http://mlrun-api:8080
> 2021-09-14 00:01:27,712 [info] Job is running in the background, pod: training-ccnqh
> 2021-09-14 00:01:51,725 [info] best iteration=1, used criteria max.accuracy
> 2021-09-14 00:01:52,403 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-v3-alexz,...5ad3e8a2,0,Sep 14 00:01:30,completed,training,v3io_user=alexzkind=jobowner=alexz,dataset,label_column=label,best_iteration=1accuracy=0.9934805156319455test-error=0.006519484368054527rocauc=0.9527879002598103brier_score=0.005873210846051267f1-score=0.6986301369863014precision_score=0.8947368421052632recall_score=0.5730337078651685,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-09-14 00:01:57,005 [info] run executed, status=completed


## Step 4 - Perform Feature Selection

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [10]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')
# feature_selection_fn.apply(auto_mount())

feature_selection_run = feature_selection_fn.run(
            params={'sample_ratio':0.25,
                    'output_vector_name':fv_name + "-short",
                   'ignore_type_errors': True},
    
            inputs={'df_artifact': transactions_fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2021-09-14 00:01:57,108 [info] starting run feature_extraction uid=54d6bedbd4e24382ad6274350c09c34c DB=http://mlrun-api:8080
> 2021-09-14 00:01:57,297 [info] Job is running in the background, pod: feature-extraction-s2rg4
> 2021-09-14 00:02:04,774 [info] Couldn't calculate chi2 because of: Input X must be non-negative.
> 2021-09-14 00:02:09,339 [info] votes needed to be selected: 3
> 2021-09-14 00:02:10,060 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-v3-alexz/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2021-09-14T00:02:10.060751+00:00', 'size': 785462}
> 2021-09-14 00:02:10,199 [info] run executed, status=completed
Pass k=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
Liblinear failed to converge, increase the number of iterations.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-v3-alexz,...0c09c34c,0,Sep 14 00:02:01,completed,feature_extraction,v3io_user=alexzkind=jobowner=alexzhost=feature-extraction-s2rg4,df_artifact,sample_ratio=0.25output_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-v3-alexz/transactions-fraud-short,f_classifmutual_info_classiff_regressionLinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2021-09-14 00:02:12,550 [info] run executed, status=completed


In [11]:
mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,label
49995,43.21,73.90,2.0,36.95,43.21,0
49996,28.89,28.89,1.0,28.89,28.89,0
49997,32.67,32.67,1.0,32.67,63.77,0
49998,16.26,16.26,1.0,16.26,16.26,0
49999,9.75,9.75,1.0,9.75,17.19,0


## Step 5 - Retrain our models with top features

In [ ]:
# why

In [13]:
# Defining our training task, including our feature vector, label and hyperparams definitions
ensemble_train_task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
ensemble_train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

classifier_fn.run(ensemble_train_task)

> 2021-09-14 00:02:25,153 [info] starting run training uid=ff23f6067a4148ca8ae90cde1252fe32 DB=http://mlrun-api:8080
> 2021-09-14 00:02:25,327 [info] Job is running in the background, pod: training-97ct8
> 2021-09-14 00:02:38,909 [info] best iteration=3, used criteria max.accuracy
> 2021-09-14 00:02:39,466 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-v3-alexz,...1252fe32,0,Sep 14 00:02:28,completed,training,v3io_user=alexzkind=jobowner=alexz,dataset,label_column=label,best_iteration=3accuracy=0.992251461988304test-error=0.0077485380116959065rocauc=0.9670901728567074brier_score=0.21354776418118845f1-score=0.65359477124183precision_score=0.7352941176470589recall_score=0.5882352941176471,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_results


> 2021-09-14 00:02:44,735 [info] run executed, status=completed


## Done!

You've completed Part 2 of the model traininig with the feature store.
Proceed to [Part 3](3-model-serving.ipynb) to learn how to deploy and monitor the model.